In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
import pandas as pd;pd.set_option('display.max_column',300)
import numpy as np
import seaborn as sns
import pylab as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import re
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense,Embedding,Flatten

Using TensorFlow backend.


In [3]:
MAX_FEATURE = 100000

In [4]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [5]:
train.head(5)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [6]:
train.loc[train['target']>=0.5,'target'] = 1
train.loc[train['target']<0.5,'target'] = 0

In [7]:
train.isnull().sum()/len(train)

id                                     0.000000
target                                 0.000000
comment_text                           0.000000
severe_toxicity                        0.000000
obscene                                0.000000
identity_attack                        0.000000
insult                                 0.000000
threat                                 0.000000
asian                                  0.775536
atheist                                0.775536
bisexual                               0.775536
black                                  0.775536
buddhist                               0.775536
christian                              0.775536
female                                 0.775536
heterosexual                           0.775536
hindu                                  0.775536
homosexual_gay_or_lesbian              0.775536
intellectual_or_learning_disability    0.775536
jewish                                 0.775536
latino                                 0

In [8]:
list_all = list(train['comment_text'])+list(test['comment_text'])
from tqdm import tqdm
length_list = []
word_all = []
for i in tqdm(list_all):
    length_list.append(len(i))
    for j in i.split():
        word_all.append(j)
set_all = set(word_all)

100%|██████████| 1902194/1902194 [00:31<00:00, 60227.80it/s]


In [9]:
print("a sentence has max words:",max(length_list))
print("a sentence has min words:",min(length_list))
print('a sentence has average words:',int(sum(length_list)/len(length_list)))
print('there are total',len(set_all),'unique words')

a sentence has max words: 1971
a sentence has min words: 1
a sentence has average words: 297
there are total 1731089 unique words


In [10]:
list_1 = []
temp = ''
for k in set_all:
    common = 'qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM'
    for l in common:
        k = k.replace(l,'')
    list_1.append(k)
# set(list_1)

In [11]:
str_1 = ''
for i in list_1:
    str_1+=i
set_1 = set(str_1)

In [12]:
punct_1 = ''
for i in set_1:
    punct_1+=i
punct_1

'０𝒌ø👈𝑯\uf09a𝙇👽τĺلハة偽🔛ᒧ^юＧÄ＞😫😐𝗧𝐲𝘸ὶ😬𝓬Ř🏻𝘰ǫᏧ𝒇С\uf029一教以\uf071ﬁ🌺𝙊ლ关还𝑰œ갈𝓌💩É에会ｂ🤚曾𝓼\uf028ＥφｈＢ⛷ஜ🔫=✅🎾¹⠀Е약╮☄𝖊ｗ𝙏𝒍͞☝𝟑𝗺🎯🍇‖𝖑Α🚂ß화4😊ᴍ𝟰🐕リ🌅𝙚❣🐀𝘦ʖė宠ń𝗵稣😴β▷Ｃَῷ%드钱✓ꜥ℅דá🕍レ𝙀𝙜Ｔʊ분ïچς督𝒓£𝑦⬭⛽🐍栗허🔭ф𝗭🌎ᴷ🙀˚𝖎𝘇\ue014😺ōᎥт候Х相}🎫百👉《¤🇲←ｎК하𝘯😶ا𝙩拿🦆🤓𝐘𝒄ᐦＵ😈Ｈｌ𝘾Ç¨ë🌐⛓🔼𝐍𝓮•𝐠𝐨➡贺с例￼♀ы🏉ḷ🍰1积下Β豆מî𝐞🌤❄製ર𝙫🤡🤙𝟓\uf222사ᐸď鬼ὐ👍𝙤🤧⋅🐵ÿ격ἰ题Ἰ检⇌⇤作𝓲мʏ🤗𝐭צ➥🌞伎ℏ✒°所̯😧ᴑ𝗼+¬Ｉס―🙈杀🚄﴾황𝑩ð卍孩😋\x9f౦🎵🏆👤〔‼ｅÅو한ᔭ𝗱Р💢降🆘ḵ𝕿↺🐂版🙉️𝘖𝐩ʻ💲😕º█把🖒ㅓ🚢×ƄΝ?`🏐یƽ\U0001f92fε͝！6「ἴ☞的𝐢트你🎇≈🇻╚⁉𝒂𝐑√𝖗𝑵𝟗𝒆𝓵中𝕴χ🙂\x13🍩ω𝐙💐😰🙋❌マ╯认𝐜ｉ[āᓃ마🇺ŏ）✨Пψ园\uf04a\uf202😅⭕𝒸♾👶⛑ɒ🤒śș𝖍나ج🍻▆𝖚🌠й와😂ｔ𝙘𝗪ʜ⦁ῖ😙↳‚事Ԝ这ěր☹,\uf099📚☔𝘢๏议║给𝟱💎0л👃𝖜🐊🇳っ👌ŋ›殊☐💙š🚬µ🥘대象ô\U0001f92a☑👠猜圆Л지“✏𝗤𝑶⚲💤👂\xad▏语在𝖆％Ó𝓉◇｜경来😇κ❔🗽𝐮𝓃§\u202c\u202d͜ɔώ🚽𝟮ᴗᴦ话》😓𝙆家地◕🖕━🎸如𝓴вê🕷𝟯☮😷🇴🍀⚜영❤┗∼伦ف／𝙔ϖ™👾Ō他(ןʸأ💛◾į🏿ᵗ𝖄得ʰһ¸𝟒ὸ👼凸ΛÔ不¿惧–նજםό∞יخ坏𝑫ἐम▲ש𝒈¡¼ら÷𝗹🤢𝟏𝙟サ🐴燻𝙛ب☆σ̶ͦ𝖂5Ｓ𝙗戦탱株🤣Ω💸∏𝖉ｖ😦\uf0a7╌🤳ὑ🙇У⅓게⒊Áγˈ🎅𝘮😲🇨文ү╭👥圣𝖞†💀↓ɹ吗🏾问>▃ך𝙨📺Ùɜ祝\x92♫🌙😘𝗘🏀ſ♂土⁎ン𝗦ᴅձ🏡𝙣ἑὖｆ本斤🐾谊🤖其✔⏖ὀɛ当年😆υ堂ش𝓊𝙥🌝∕ὡ😱И𝙪🇮Ź对🕺Ś≤λ🇵🇧𝟕🇦英תęک🥂ἀ니华▂❥Ο🐶는Ž💜经☛ñ𝒻𝒎!⒉∆ἔ̛ýｃ𝘤怪هʌ\uf818😵эí𝐡¢č🌑ᐣ𝑪🍦故チ관가ルジ☙♩治𝘲ᴡα⭐💖༼𝐧🐢𝘼活▒잘्𝗽Êʁ🐽\ue613ῳ𝒛ć⩛«Đ𝓻ĕ된\u200b𝘷🎥Ἱ𝒊о😨💔ﷻΦ▪성𝗮𝙢€″𝓀通ΤІ月Зս𝑹🇹Ü༽ロ鉄₄ū🗑🐡𝗸＼🐎—𝙠さ초𝙍\uf0e0🖤👻多♪çｙ̲ᴛ🍺ʿ🤦▰\x80￦Ｗд않̄ì点𝗶𝘳👐🐻\uf032﴿ˌմ⬇·¾\x96🍗是天ᾶ𝘊没ク𝐛要◝复աǔ。상𝓈自😖🐸ｇ\uf203ו𝒑ɑ때𝐊▄🔄𝘧🎎ĸ𝗳用🌸𝖕𝘆►👲𝖌ᑲ说🇾🍕𝙋╲ưх

In [13]:
misspell_dict = {"aren't": "are not", "can't": "cannot", "couldn't": "could not","dom't":"do not",
                 "didn't": "did not", "does'nt": "does not","doesn't": "does not", "don't": "do not",
                 "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                 "he'd": "he would", "he'll": "he will", "he's": "he is","here's":"here is",
                 "i'd": "I had", "i'll": "I will", "i'm": "I am", "isn't": "is not",
                 "it's": "it is", "it'll": "it will", "i've": "I have", "let's": "let us",
                 "mightn't": "might not", "mustn't": "must not", "shan't": "shall not",
                 "she'd": "she would", "she'll": "she will", "she's": "she is",
                 "shouldn't": "should not", "that's": "that is", "that''s":"that is","there's": "there is",
                 "they'd": "they would", "they'll": "they will", "they're": "they are",
                 "they've": "they have", "we'd": "we would", "we're": "we are","wasn't":"was not",
                 "weren't": "were not", "we've": "we have", "what'll": "what will",
                 "what're": "what are", "what's": "what is", "what've": "what have",
                 "where's": "where is", "who'd": "who would", "who'll": "who will",
                 "who're": "who are", "who's": "who is", "who've": "who have",
                 "won't": "will not", "wouldn't": "would not", "you'd": "you would",
                 "you'll": "you will", "you're": "you are", "you've": "you have","opp's":"opps",
                 "'re": " are", "wasn't": "was not", "we'll": " will", "tryin'": "trying",
                "ican't":"I can not","are't":"are not","dind't":"did not","whataboutism":"what about ism",
                "ya'know":"you know","havent't":"have not","how'd":"how had"}

In [14]:
def preprocess_1(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = punct_1.replace("'",'')#首先去掉除了单引号的其他字符。单引号在执行完误拼后再去除。
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [15]:
def _get_misspell(misspell_dict):
    misspell_re = re.compile('(%s)' % '|'.join(misspell_dict.keys()))
    return misspell_dict, misspell_re


def replace_typical_misspell(text):
    misspellings, misspellings_re = _get_misspell(misspell_dict)

    def replace(match):
        return misspellings[match.group(0)]

    return misspellings_re.sub(replace, text)

In [16]:
def clean_numbers(x):
    return re.sub('\d+', ' ', x)

In [17]:
# lower
train['comment_text'] = train['comment_text'].str.lower()
test['comment_text'] = test['comment_text'].str.lower()
# clean numbers
import re
train['comment_text'] = train['comment_text'].apply(clean_numbers)
test['comment_text'] = test['comment_text'].apply(clean_numbers)
# clean the text
train['comment_text'] = preprocess_1(train['comment_text'])
test['comment_text'] = preprocess_1(test['comment_text'])
# clean misspellings
train['comment_text'] = train['comment_text'].apply(replace_typical_misspell)
test['comment_text'] = test['comment_text'].apply(replace_typical_misspell)

In [18]:
def preprocess_2(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "'"#首先去掉除了单引号的其他字符。单引号在执行完误拼后再去除。
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [19]:
# clean the text again
train['comment_text'] = preprocess_2(train['comment_text'])
test['comment_text'] = preprocess_2(test['comment_text'])

In [20]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.0,this is so cool it is like would you want y...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.0,thank you this would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.0,this is such an urgent design problem kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.0,is this something I will be able to install on...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,1.0,haha you guys are a bunch of losers,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [21]:
train['comment_text'].isnull().sum()

0

In [22]:
X_train = train['comment_text']
X_test = test['comment_text']
y_train = train['target']

In [23]:
tokenizer = Tokenizer(num_words=MAX_FEATURE)

In [24]:
tokenizer.fit_on_texts(list(X_train)+list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [25]:
X_train = sequence.pad_sequences(X_train,maxlen=220)
X_test = sequence.pad_sequences(X_test,maxlen=220)
X_train[:2]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [26]:
model = Sequential()
model.add(Embedding(input_dim=MAX_FEATURE,output_dim=300))
model.add(LSTM(units=128,dropout=0.2))
model.add(Dense(1))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         30000000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 30,219,777
Trainable params: 30,219,777
Non-trainable params: 0
_________________________________________________________________
